This gets a unix timestamp input, converts it to a pandas compatible datetime, runs it against known hourly data, and returns a temperature. Linear interpolation will be used between two hourly data points. 

Maybe some optimization could be done if the two hourly temps have changed less than a certain amount.

In [1]:
from netCDF4 import Dataset, MFDataset, num2date
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

at_data = Dataset("../data/air_temperature.nc")
air_temperature = at_data.variables['air_temperature'][:]
at_time = at_data.variables['time'][:]

In [2]:
time_var = at_data.variables['time']
dates = num2date(at_time[:], time_var.units)
dates_pd = pd.to_datetime(dates)

In [3]:
df = pd.DataFrame(dates_pd,columns=['Date'])
df['air_temperature'] = air_temperature[:]
df = df.set_index('Date')

In [9]:
pumaRange = df.loc['2018-10':'2019-05']
type(pumaRange.iloc[-1].name.to_pydatetime())

datetime.datetime

The way the algorithm will work is that it will check the timestamp that is input and see if it is +/- 15 minutes from the top of the hour. If it is, then the returned temperature will just be that hourly datapoint. If it is not, then two weather points before and after the input time, then the returned temperature will be an interpolation between the two

In [17]:
def outdoorTemp(inUnix):
    #BEWARE: incoming data is offset from UTC by -8 hours
    #or 28,800 seconds
    pumaPoint = datetime.utcfromtimestamp(inUnix - 28800)
   
    dataStart = pumaRange.iloc[0].name.to_pydatetime()
    dataEnd = pumaRange.iloc[-1].name.to_pydatetime()
    
    if pumaPoint < dataStart or pumaPoint > dataEnd:
        raise Exception('Error: date provided out of bounds')
    nearestIndex = pumaRange.index.get_loc(pumaPoint, method='nearest')
    
    #print(nearestIndex)
    nearestTemp = pumaRange.iloc[nearestIndex]
    #print(str(nearestTemp.name) + ': ' + str(nearestTemp.air_temperature) + 'F')

    timeDiff = nearestTemp.name - pumaPoint #timeDelta
    #in this expression, when the point is ahead, the day=0
    #when the point is behind, the day=-1
    #print(timeDiff)
    
    if abs(timeDiff) < timedelta(minutes = 15):
        #less than 15 minutes from an hour
        return(nearestTemp.air_temperature)
    elif timeDiff.days == -1:
        #greater than 15 minutes from the hour
        #and goes to previous hour
        #so look forward to interpolate
        
        #have to convert pandas timestamp to datetime to python timestamp
        xpRaw = [nearestTemp.name.to_pydatetime(), pumaRange.iloc[nearestIndex + 1].name.to_pydatetime()]
        xp = np.array([time.mktime(d.timetuple()) for d in xpRaw])
        
        fp = [nearestTemp.air_temperature, pumaRange.iloc[nearestIndex + 1].air_temperature]
        #print(type(pumaPoint))
        
        return(round(np.interp(time.mktime(pumaPoint.timetuple()), xp, fp), 2))
    else:
        #greater than 15 minutes from the hour
        #and goes to next hour
        #so look backward to interpolate
        
        xpRawToo = [pumaRange.iloc[nearestIndex - 1].name.to_pydatetime(), nearestTemp.name.to_pydatetime()]
        xpToo = np.array([time.mktime(d.timetuple()) for d in xpRawToo])
        
        fpToo = [pumaRange.iloc[nearestIndex - 1].air_temperature, nearestTemp.air_temperature]
        
        return(round(np.interp(time.mktime(pumaPoint.timetuple()), xpToo, fpToo), 2))

print(outdoorTemp(1500000000))

Exception: Error: date provided out of bounds

In [18]:
print(outdoorTemp(1546386804)) #2019-1-1 23:53:24 goes to the next hour, less that 15 min
print('\r\n')
print(outdoorTemp(1546381404)) #2019-1-1 22:23:24 goes to the previous hour, more than 15
print('\r\n')
print(outdoorTemp(1546380612)) #2019-1-1 22:10:12 goes to the previous hour, less than 15
print('\r\n')
print(outdoorTemp(1546382112)) #2019-1-1 22:35:12 goes to the next hour, more than 15

31.82


28.78


25.34


30.51
